# Unions of two vector files for Vienna in EPSG 31256 projection

The following script helps to produce transfer vector information into the 100m Vienna GRID




# 1) The next script transformed the "Bezirke" into 100 m cells - every 100m gets the covering "Bezirke" Name

In [2]:
# geometry:
import geopandas as gpd

# 1 Load GRID
shp_100m_GRID = gpd.read_file(r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\reference_units\GRID\grid31256\fishnet_100m_31256.shp")
# 2 LOAD "Bezirke"
shp = gpd.read_file(r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\reference_units\Bezirke\Bezirke_31256.shp")

# Perform spatial union
union_result = gpd.overlay(shp_100m_GRID, shp, how='union')

# Save the result to a new shapefile
union_result.to_file(r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\100m_GRID\Bezirke_100m.shp")

print ("done")

done


Alternative: ZONAL JOIN overlay the Bezirke-layer with the 100m GRID:

In [11]:
## zonal join:

import geopandas as gpd

# Load the point and polygon shapefiles
points = gpd.read_file(r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\reference_units\GRID\grid31256\fishnet_100m_31256_points.shp")


polygons = gpd.read_file(r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\reference_units\Bezirke\Bezirke_31256.shp")

output_shp = r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\100m_GRID\Bezirke_100m_points.shp"

# Perform the zonal join
joined_data = gpd.sjoin(points, polygons, how="left", op="within")

# 'left' indicates that we want to keep all points, even if they don't intersect with any polygon
# 'op="within"' indicates that we want to find points that fall within the polygons

# Remove points where index_right is NaN
joined_data = joined_data.dropna(subset=['index_right'])


# Display or manipulate the joined data as needed
#print(joined_data.head())

# Save the joined data to a new shapefile if necessary
joined_data.to_file(output_shp)
print ("done")

c:\ProgramData\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3400: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
C:\Users\ManuelLöhnertz\AppData\Local\Temp\ipykernel_28576\3281548056.py:25: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  joined_data.to_file(output_shp)


done


# 2) If needed, the following code produce a raster from the grid

In [20]:
#https://gist.github.com/perrygeo/721040f8545272832a42

import rasterio
import fiona
import numpy as np
from rasterio import features
from affine import Affine
from shapely.geometry import shape, box



################################################################## INPUT

input_vector_polygon = r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\100m_GRID\Bezirke_100m.shp"
#resolution = 100 # pixel size in m
output_raster_tif = r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\100m_GRID\Bezirke_100m2.tif"

attribute_name = 'BEZNR'

##################################################################


def _rasterize_geom(geom, shape, affinetrans, all_touched):
    indata = [(geom, 1)]
    rv_array = features.rasterize(
        indata,
        out_shape=shape,
        transform=affinetrans,
        fill=0,
        all_touched=all_touched)
    return rv_array


def rasterize_pctcover(geom, atrans, shape):
    alltouched = _rasterize_geom(geom, shape, atrans, all_touched=True)
    exterior = _rasterize_geom(geom.exterior, shape, atrans, all_touched=True)

    # Create percent cover grid as the difference between them
    # at this point all cells are known 100% coverage,
    # we'll update this array for exterior points
    pctcover = (alltouched - exterior) * 100

    # loop through indicies of all exterior cells
    for r, c in zip(*np.where(exterior == 1)):

        # Find cell bounds, from rasterio DatasetReader.window_bounds
        window = ((r, r+1), (c, c+1))
        ((row_min, row_max), (col_min, col_max)) = window
        x_min, y_min = (col_min, row_max) * atrans
        x_max, y_max = (col_max, row_min) * atrans
        bounds = (x_min, y_min, x_max, y_max)

        # Construct shapely geometry of cell
        cell = box(*bounds)

        # Intersect with original shape
        cell_overlap = cell.intersection(geom)

        # update pctcover with percentage based on area proportion
        coverage = cell_overlap.area / cell.area
        pctcover[r, c] = int(coverage * 100)

    return pctcover


if __name__ == "__main__":
    with fiona.open(input_vector_polygon) as src:
        geom = shape(next(src)['geometry'])  # take first feature's shapely geometry

    scale = 100
    atrans = Affine(1.0 / scale, 0.0, 0.0, 0.0, -1.0 / scale, 5.0)
    shape = (5 * scale, 5 * scale)
    profile = {
        'affine': atrans,
        'height': shape[0],
        'width': shape[1],
        'count': 1,
        'crs': {},
        'driver': 'GTiff',
        'dtype': 'uint8',
        'nodata': None,
        'tiled': False}

    pctcover = rasterize_pctcover(geom, atrans=atrans, shape=shape)
    with rasterio.open(output_raster_tif, 'w', **profile) as dst:
        dst.write(pctcover, 1)


print ("done")

done


C:\Users\ManuelLöhnertz\AppData\Local\Temp\ipykernel_28576\496379565.py:68: FionaDeprecationWarning: Collection.__next__() is buggy and will be removed in Fiona 2.0. Switch to `next(iter(collection))`.
  geom = shape(next(src)['geometry'])  # take first feature's shapely geometry
C:\Users\ManuelLöhnertz\AppData\Roaming\Python\Python310\site-packages\rasterio\__init__.py:331: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = writer(
